In [16]:
import numpy as np
import torch
import sys
sys.path.insert(1, './PSM_V1')
from sobolev import Sobolev
from sobolev import Sobolev
from solver import Solver
from utils import matmul
import surrogates
import time
from diffeomorphisms import hyper_rect
import matplotlib
import matplotlib.pyplot as plt
import scipy
#style.use('dark_background')
matplotlib.rcdefaults() 

### Initialize Data 

In [2]:
%%time
start = time.time()
deg_quad = 50
rect = rect = np.array([[-1.0, 1.0], [-1.0, 1.0]])
diffeo_param = hyper_rect(*rect)
sob_param = Sobolev(deg=deg_quad, dim=2)
##### Sobolev norm for the inteprolation
sob_param.set_s(0)
#####
x_plt, _, _, x, _, _ = sob_param.get_quad()
metric_param = sob_param.metric()
W_param = sob_param.get_leja_weights()
u_ob = surrogates.Polynomial(n=deg_quad,p=np.inf, dim=2)
metric_2d = sob_param.metric(weak=True)
x_l = sob_param.get_xs()
x = x_l[0][0]#np.linspace(-1,1,10)
#Create Regression Matrix
X_p = u_ob.data_axes([x,x]).T
X_plg = u_ob.data_axes([x,x]).T
end = time.time()

CPU times: user 11.1 s, sys: 1.75 s, total: 12.9 s
Wall time: 3.06 s


## Kernel $\eta_\epsilon(x-x_i)$ of order 4

In [3]:
def K_1(x,a ,e): 
    if a<x<=a+e:
        return (2/3-9/2*((x-a)/e)**2+19/3*((x-a)/e)**3-5/2*((x-a)/e)**4)
    elif a-e<x<=a:
        return (2/3-9/2*((x-a)/e)**2-19/3*((x-a)/e)**3-5/2*((x-a)/e)**4)
    else: 
        return 0
#def K_2(x,a ,e): 
#    if a<x<=a+e:
#        return (2/3-9/2*((x-a)/e)**2+19/3*((x-a)/e)**3-5/2*((x-a)/e)**4)**2
#    elif a-e<x<=a:
#        return (2/3-9/2*((x-a)/e)**2-19/3*((x-a)/e)**3-5/2*((x-a)/e)**4)**2
#    else: 
        return 0
def K_p(x,a,b,e):
    return K_1(x,a,e)*K_1(x,b,e)

In [4]:
from scipy.integrate import quad
a= 0.3
b = 0.25
e = 1
N=np.array([quad(K_1,-1,1,args=(a,e))[0] for a in np.linspace(-1,1,10)])

In [5]:
def MSE_reg(n_points, deg_quad):
    u_ob = surrogates.Polynomial(n=deg_quad,p=np.inf, dim=1)
    x_p = np.linspace(-1,1,n_points)
    idx = np.random.randint(1,2*n_points,n_points)
    x = x_p#x_p[idx]
    X_p = u_ob.data_axes([x]).T
    #Compute 
    K = np.matmul(X_p.T,X_p)
    f = np.matmul(X_p.T,gt(x))
    #reg_coeff,_,_,_ = scipy.linalg.lstsq(K,f)
    reg_coeff = torch.matmul(K.inverse(),f)
    u_ob.set_weights(torch.tensor(reg_coeff))
    return u_ob

In [11]:
def sph_reg_full(n_points, deg_quad):
    # Compute Vandermorde matrix X_p for the regression
    u_ob = surrogates.Polynomial(n=deg_quad,p=np.inf, dim=1)
    x_p = np.linspace(-1,1,n_points)
    #idx = np.random.randint(1,2*n_points,n_points)
    x = x_p#x_p[idx]
    dx = abs(x[0]-x[1])
    X_p = u_ob.data_axes([x]).T
    #####
    e = 4*dx
    #N_m = np.diag(N(x,e,dx))
    M_sph = np.array([[quad(K_p,-1,1,args=(a,b,e))[0] for a in x]for b in x])*dx**2
    #Compute 
    K = np.matmul(np.matmul(X_p.T,M_sph),X_p)
    f = np.matmul(np.matmul(X_p.T,M_sph),gt(x))
    reg_coeff,_,_,_ = scipy.linalg.lstsq(K,f)
    #reg_coeff = torch.matmul(K.inverse(),f)
    u_ob.set_weights(torch.tensor(reg_coeff))
    return u_ob

In [18]:
def sph_reg_diag(n_points, deg_quad):
    u_ob = surrogates.Polynomial(n=deg_quad,p=np.inf, dim=1)
    x_p = np.linspace(-1,1,n_points)
    idx = np.random.randint(1,n_points,n_points)
    x = x_p#x_p[idx]
    dx = abs(x[0]-x[1])
    X_p = u_ob.data_axes([x]).T
    e = 4*dx
    #N_m = np.diag(N(x,e,dx))
    M_sph = np.diag(np.array([quad(K_1,-1,1,args=(a,e))[0] for a in x]))*dx**2
    #Compute 
    K = np.matmul(np.matmul(X_p.T,M_sph),X_p)
    f = np.matmul(np.matmul(X_p.T,M_sph),gt(x))
    reg_coeff,_,_,_ = scipy.linalg.lstsq(K,f)
    #reg_coeff = torch.matmul(K.inverse(),f)
    u_ob.set_weights(torch.tensor(reg_coeff))
    return u_ob

### Define ground truth 

In [13]:
def gt(x):
    return x**3*np.tanh(10*(x-np.pi))+np.cos(15*x)

### Compute errors for SPH and MSE

In [14]:
x_t = np.linspace(-0.9,0.9,1000)
deg_model = 50
X_tr = torch.tensor(MSE_reg(4, deg_model).data_axes([x_t]).T, dtype = torch.float32)
e_mse = [ np.mean(abs(MSE_reg(k, 50)(X_tr).T[0].detach().numpy()-gt(x_t))) for k in range(deg_model-20,100)]

/tmp/ipykernel_138970/3205795487.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u_ob.set_weights(torch.tensor(reg_coeff))
/tmp/ipykernel_138970/2128255700.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tr = torch.tensor(MSE_reg(4, deg_model).data_axes([x_t]).T, dtype = torch.float32)


In [19]:
e_sph = [np.mean(abs(sph_reg_full(k, deg_model)(X_tr).T[0].detach().numpy()-gt(x_t))) 
           for k in range(deg_model-20,100)]
e_sph_diag = [np.mean(abs(sph_reg_diag(k, deg_model)(X_tr).T[0].detach().numpy()-gt(x_t))) 
           for k in range(deg_model-20,100)]

/tmp/ipykernel_138970/1860876543.py:12: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  M_sph = np.array([[quad(K_p,-1,1,args=(a,b,e))[0] for a in x]for b in x])*dx**2


In [ ]:
N_d =np.linspace(2,len(e_mse)*2,(len(e_mse)))
plt.loglog(N_d,e_mse, label ='MSE Regression');
plt.loglog(N_d,e_sph, label ='SPH Regression');
plt.loglog(N_d,e_sph_diag, label ='SPH Regression');
#plt.loglog(N_d[0:len(np.array(e_sph_4))], e_sph_4, label ='SPH_loss_4');
#plt.loglog(N_d, (1/N_d)**5, label ='O(50)')
plt.legend()